Esto es una pprueba

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# ============================================================
# 1. CONFIGURACIÓN DE GPU
# ============================================================
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar para que no reserve toda la VRAM de golpe
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU DETECTADA: {gpus}")
        print(f"🚀 Versión de TensorFlow: {tf.__version__}")
    except RuntimeError as e:
        print(f"⚠️ Error configurando GPU: {e}")
else:
    print("❌ NO SE DETECTÓ GPU. El entrenamiento será más lento.")

# ============================================================
# 2. CONFIGURACIÓN DE RUTAS Y CÓDIGO LOCAL
# ============================================================
# Añadimos la raíz del proyecto al path para poder importar 'educational_ai_analytics'
root_path = os.path.abspath("..")
if root_path not in sys.path:
    sys.path.insert(0, root_path)

# Limpiamos caché del módulo para asegurar que cargamos los últimos cambios del .py
if 'educational_ai_analytics' in sys.modules:
    del sys.modules['educational_ai_analytics']
if 'educational_ai_analytics.modeling' in sys.modules:
    del sys.modules['educational_ai_analytics.modeling']

print(f"📍 Raíz del proyecto añadida: {root_path}")
print("✨ Celda de configuración completada.")

2026-02-13 15:38:41.746435: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/usr

✅ GPU DETECTADA: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
🚀 Versión de TensorFlow: 2.20.0
📍 Raíz del proyecto añadida: /workspace/TFM_education_ai_analytics
✨ Celda de configuración completada.


In [2]:
# ============================================================
# 3. CARGA DE DATOS Y CONSTRUCCIÓN DE TENSORES
# ============================================================
DATA_DIR = Path("../data/processed/training/features")

def prepare_tensors(path: Path):
    print(f"🚀 Cargando features desde: {path}")
    
    # Cargamos los CSVs asegurando alineación total mediante sort_index()
    # (Si no se ordenan, el alumno de la fila 1 de clics podría ser el de la fila 50 de static)
    clicks = pd.read_csv(path / "ts_clicks.csv", index_col=0).sort_index()
    perf = pd.read_csv(path / "ts_performance.csv", index_col=0).sort_index()
    proc = pd.read_csv(path / "ts_procrastination.csv", index_col=0).sort_index()
    static = pd.read_csv(path / "static_features.csv", index_col=0).sort_index()
    
    n_students = len(clicks)
    print(f"✅ Estudiantes cargados: {n_students}")

    def to_3d_tensor(df, num_vars):
        # Convierte Tabla Ancha (N, Vars*40) -> Tensor 3D (N, 40, Vars)
        return df.values.reshape(n_students, num_vars, 40).transpose(0, 2, 1)

    # Transformamos cada bloque a 3D
    tensor_clicks = to_3d_tensor(clicks, 4)
    tensor_perf = to_3d_tensor(perf, 6)
    tensor_proc = to_3d_tensor(proc, 1)

    # Concatenamos todas las variables temporales (Total: 4+6+1 = 11 variables)
    X_temp_raw = np.concatenate([tensor_clicks, tensor_perf, tensor_proc], axis=2)
    X_static = static.values
    
    return X_temp_raw, X_static

# 1. Ejecutamos la carga limpia desde el disco
X_train_temp_raw, X_train_static = prepare_tensors(DATA_DIR)

# 2. NORMALIZACIÓN GLOBAL (Clave para que PCA y AE compitan igual)
# Aplicamos MinMaxScaler a las 11 variables temporales a la vez
N, T, F = X_train_temp_raw.shape
scaler = MinMaxScaler()
# Aplanamos, entrenamos el scaler y volvemos a dar forma 3D
X_train_temp = scaler.fit_transform(X_train_temp_raw.reshape(-1, F)).reshape(N, T, F)

print("-" * 60)
print(f"📊 Tensor Temporal (X_train_temp):   {X_train_temp.shape}")
print(f"📊 Tensor Estático (X_train_static): {X_train_static.shape}")
print(f"✅ Rango 0-1 verificado: {X_train_temp.min()} a {X_train_temp.max()}")
print(f"✨ Datos listos para el entrenamiento.")

🚀 Cargando features desde: ../data/processed/training/features
✅ Estudiantes cargados: 22785
------------------------------------------------------------
📊 Tensor Temporal (X_train_temp):   (22785, 40, 11)
📊 Tensor Estático (X_train_static): (22785, 10)
✅ Rango 0-1 verificado: 0.0 a 1.0
✨ Datos listos para el entrenamiento.


### BASELINE: PCA

In [3]:
# ============================================================
# 4. BASELINE: PCA (Reducción Lineal)
# ============================================================
from educational_ai_analytics.modeling.params import AE_PARAMS

# Sacamos la dimensión latente directamente de tu archivo central de parámetros
latent_dim = AE_PARAMS.latent_dim

# Preparamos los datos aplanados para el PCA
X_train_flat = X_train_temp.reshape(X_train_temp.shape[0], -1)
X_pca_input = np.concatenate([X_train_flat, X_train_static], axis=1)

print(f"📏 Usando latent_dim de params.py: {latent_dim}")

# Entrenamos PCA
pca = PCA(n_components=latent_dim, random_state=42)
latent_pca = pca.fit_transform(X_pca_input)
reconstructed_pca = pca.inverse_transform(latent_pca)

# Calculamos el MSE
mse_pca = np.mean(np.square(X_pca_input - reconstructed_pca))

print("-" * 60)
print(f"📉 MSE del PCA ({latent_dim} dims): {mse_pca:.6f}")
print(f"📈 Varianza explicada total: {np.sum(pca.explained_variance_ratio_):.4f}")
print("-" * 60)

📏 Usando latent_dim de params.py: 32
------------------------------------------------------------
📉 MSE del PCA (32 dims): 0.005180
📈 Varianza explicada total: 0.9048
------------------------------------------------------------


In [4]:
# ============================================================
# 5. ENTRENAMIENTO DINÁMICO DESDE PARAMS.PY
# ============================================================
from educational_ai_analytics.modeling import TimeSeriesAutoencoder
from educational_ai_analytics.modeling.params import AE_PARAMS
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Instanciamos usando TODA la configuración centralizada
model = TimeSeriesAutoencoder(
    latent_dim=AE_PARAMS.latent_dim,
    timesteps=AE_PARAMS.timesteps,
    temp_features=AE_PARAMS.temp_features,
    static_features=AE_PARAMS.static_features,
    lstm_units=AE_PARAMS.lstm_units,
    dense_static_units=AE_PARAMS.dense_static_units,
    activation=AE_PARAMS.activation
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=AE_PARAMS.learning_rate), 
    loss='mse'
)

# Callbacks configurados desde params.py
callbacks = [
    ModelCheckpoint(
        "../models/best_autoencoder.keras", 
        save_best_only=True, 
        monitor='val_loss'
    ),
    EarlyStopping(
        monitor='val_loss', 
        patience=AE_PARAMS.early_stopping_patience, 
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss', 
        factor=AE_PARAMS.reduce_lr_factor, 
        patience=AE_PARAMS.reduce_lr_patience, 
        min_lr=AE_PARAMS.min_learning_rate,
        verbose=1
    )
]

history = model.fit(
    x=[X_train_temp, X_train_static],
    y=X_train_temp, 
    epochs=AE_PARAMS.epochs,
    batch_size=AE_PARAMS.batch_size,
    validation_split=AE_PARAMS.validation_split,
    callbacks=callbacks,
    verbose=1
)

I0000 00:00:1770997129.208547   67423 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5580 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/150


2026-02-13 15:38:54.727763: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91900


143/143 ━━━━━━━━━━━━━━━━━━━━ 12s 47ms/step - loss: 0.0257 - val_loss: 0.0131 - learning_rate: 0.0010
Epoch 2/150
143/143 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - loss: 0.0121 - val_loss: 0.0103 - learning_rate: 0.0010
Epoch 3/150
143/143 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - loss: 0.0104 - val_loss: 0.0097 - learning_rate: 0.0010
Epoch 4/150
143/143 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - loss: 0.0096 - val_loss: 0.0089 - learning_rate: 0.0010
Epoch 5/150
143/143 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - loss: 0.0091 - val_loss: 0.0086 - learning_rate: 0.0010
Epoch 6/150
143/143 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - loss: 0.0088 - val_loss: 0.0082 - learning_rate: 0.0010
Epoch 7/150
143/143 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - loss: 0.0085 - val_loss: 0.0079 - learning_rate: 0.0010
Epoch 8/150
143/143 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - loss: 0.0082 - val_loss: 0.0078 - learning_rate: 0.0010
Epoch 9/150
143/143 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - loss: 0.0079 - val_loss: 0.0074 - learning_rate: 0.0010
Epo

In [ ]:
# ============================================================
# 6. EXTRACCIÓN Y GUARDADO DE EMBEDDINGS (ESPACIO LATENTE)
# ============================================================
import os

# 1. Crear directorio
output_path = "../data/processed/training/embeddings"
os.makedirs(output_path, exist_ok=True)

# 2. Obtener IDs de alumnos (los guardamos al cargar en la Celda 2)
# Si perdiste el índice, cárgalo de uno de los CSVs originales
student_ids = pd.read_csv(DATA_DIR / "static_features.csv", index_col=0).index

# 3. Extraer espacio latente del Autoencoder
# Usamos la función .encode() que definimos en el modelo
latent_ae_values = model.encode([X_train_temp, X_train_static]).numpy()

# 4. Crear DataFrames con los IDs como índice
df_latent_ae = pd.DataFrame(
    latent_ae_values, 
    index=student_ids, 
    columns=[f"ae_dim_{i}" for i in range(AE_PARAMS.latent_dim)]
)

df_latent_pca = pd.DataFrame(
    latent_pca, 
    index=student_ids, 
    columns=[f"pca_dim_{i}" for i in range(AE_PARAMS.latent_dim)]
)

# 5. Guardar en CSV
df_latent_ae.to_csv(f"{output_path}/latent_ae.csv")
df_latent_pca.to_csv(f"{output_path}/latent_pca.csv")

print(f"✅ Embeddings guardados en: {output_path}")
print(f"📊 Dimensiones AE:  {df_latent_ae.shape}")
print(f"📊 Dimensiones PCA: {df_latent_pca.shape}")